<a href="https://colab.research.google.com/github/RahafSobh/RahafSobh/blob/main/hw4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.preprocessing import normalize
import random

def generate_tfidf_documents(num_docs=15, random_seed=42):
    """
    מחזיר DataFrame של מסמכים (שורות) ו-terms (עמודות)
    עם ציוני tf-idf אקראיים כמו בדרישה.
    """
    random.seed(random_seed)
    np.random.seed(random_seed)

    vocab = ["team", "coach", "hockey", "baseball", "soccer",
             "penalty", "score", "win", "loss", "season"]

    data = []

    for doc_id in range(num_docs):
        row = []
        for term in vocab:
            # הגרלה אם המילה מופיעה בטקסט
            appears = random.choice([0, 1])
            if appears == 0:
                tfidf = 0.0
            else:
                # ציון idf אקראי בין 2 ל-6 (כולל עשרוני)
                tfidf = np.round(np.random.uniform(2.0, 6.0), 3)
            row.append(tfidf)
        data.append(row)

    df = pd.DataFrame(data, columns=vocab)
    df.index = [f"doc_{i+1}" for i in range(num_docs)]
    return df

def kmeans_cosine_clustering(df, k=2, random_seed=42):
    """
    מריץ K-means עם מדד שקול לדמיון קוסינוס:
    מנרמלים את הווקטורים לאורך 1 ואז משתמשים ב-KMeans רגיל
    (אוקלידי על וקטורים מנורמלים).
    """
    # נרמול L2 של המסמכים (שורות)
    X = df.values
    X_normalized = normalize(X, norm='l2', axis=1)

    kmeans = KMeans(n_clusters=k, random_state=random_seed, n_init=10)
    kmeans.fit(X_normalized)

    labels = kmeans.labels_
    centers = kmeans.cluster_centers_

    return labels, centers

def print_top_features_per_cluster(centers, feature_names, top_n=5):
    """
    מדפיס את חמשת המאפיינים המשמעותיים ביותר בכל אב-טיפוס (centroid)
    ואת מספר ה-cluster.
    """
    num_clusters = centers.shape[0]
    for cluster_idx in range(num_clusters):
        centroid = centers[cluster_idx]
        # אינדקסים של התכונות מהגדול לקטן
        top_indices = np.argsort(centroid)[::-1][:top_n]
        top_terms = [(feature_names[i], np.round(centroid[i], 3)) for i in top_indices]

        print(f"\n=== Cluster {cluster_idx} ===")
        print("Top features (term, value):")
        for term, val in top_terms:
            print(f"  {term}: {val}")

def main():
    # שלב א' – יצירת הקלט
    print("Generating synthetic tf-idf document matrix...")
    df_docs = generate_tfidf_documents(num_docs=15, random_seed=42)

    print("\nDocument-term matrix (tf-idf):")
    print(df_docs)

    # שלב ב' – K-means עם K=2 ודמיון קוסינוס (באמצעות נרמול)
    print("\nRunning K-means clustering with K=2 (cosine-based)...")
    labels, centers = kmeans_cosine_clustering(df_docs, k=2, random_seed=42)

    # הוספת תווית ה-cluster לכל מסמך
    df_with_clusters = df_docs.copy()
    df_with_clusters["cluster"] = labels

    print("\nDocuments with assigned clusters:")
    print(df_with_clusters)

    # שלב ג' – הצגת 5 המאפיינים המשמעותיים בכל אב-טיפוס
    print("\nTop 5 features for each cluster prototype:")
    print_top_features_per_cluster(centers, df_docs.columns, top_n=5)

if __name__ == "__main__":
    main()


Generating synthetic tf-idf document matrix...

Document-term matrix (tf-idf):
         team  coach  hockey  baseball  soccer  penalty  score    win   loss  \
doc_1   0.000  0.000   3.498     0.000   0.000    0.000  0.000  0.000  5.803   
doc_2   0.000  0.000   0.000     0.000   0.000    0.000  4.928  0.000  4.395   
doc_3   0.000  0.000   2.624     2.232   5.465    0.000  0.000  4.404  0.000   
doc_4   4.832  0.000   2.082     5.880   5.330    0.000  2.849  0.000  2.727   
doc_5   2.734  3.217   0.000     0.000   0.000    0.000  4.099  0.000  0.000   
doc_6   3.728  3.165   4.447     2.558   0.000    3.169  3.465  0.000  3.824   
doc_7   0.000  0.000   0.000     5.141   2.799    4.057  0.000  4.370  0.000   
doc_8   0.000  2.186   4.430     2.682   0.000    0.000  2.260  0.000  5.796   
doc_9   5.234  0.000   3.218     0.000   0.000    2.391  4.737  3.761  2.488   
doc_10  0.000  3.981   0.000     0.000   0.000    0.000  0.000  2.138  0.000   
doc_11  3.035  4.650   3.247     0.000   